In [2]:
let a: u32 = 10;
a

10

# Pattern Matching

## Let Control Flow

### if-let

In [3]:
fn sleep_for(secs: f32) {
    let dur = if let Ok(dur_) = std::time::Duration::try_from_secs_f32(secs) {
        dur_
    } else {
        std::time::Duration::from_millis(500)
    };
    std::thread::sleep(dur);
    println!("slept for {:?}", dur);
}

In [4]:
sleep_for(-10.0);
sleep_for(0.8);
sleep_for(0.5);

slept for 500ms
slept for 800.000012ms
slept for 500ms


### let-else

In [5]:
#[derive(Debug)]
enum Result {
    Ok(u32),
    Err(String),
}

fn hex_or_die_trying(maybe_string: Option<String>) -> Result {
    let s = if let Some(s) = maybe_string {
        s
    } else {
        return Result::Err(String::from("got None"));
    };

    let first_byte_char = if let Some(first_byte_char) = s.chars().next() {
        first_byte_char
    } else {
        return Result::Err(String::from("got empty string"));
    };

    if let Some(digit) = first_byte_char.to_digit(16) {
        Result::Ok(digit)
    } else {
        Result::Err(String::from("not a hex digit"))
    }
}

In [6]:
println!("result: {:?}", hex_or_die_trying(Some(String::from("foo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from("Aoo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from(""))));
println!("result: {:?}", hex_or_die_trying(None));

result: Ok(15)
result: Ok(10)
result: Err("got empty string")
result: Err("got None")


#### neat version

In [7]:
fn hex_or_die_trying_flatten(maybe_string: Option<String>) -> Result {
    let Some(s) = maybe_string else {
        return Result::Err(String::from("got None"));
    };

    let Some(first_byte_char) = s.chars().next() else {
        return Result::Err(String::from("got empty string"));
    };

    let Some(digit) = first_byte_char.to_digit(16) else {
        return Result::Err(String::from("not a hex digit"));
    };

    Result::Ok(digit)
}

In [8]:
println!("result: {:?}", hex_or_die_trying(Some(String::from("foo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from("Aoo"))));
println!("result: {:?}", hex_or_die_trying(Some(String::from(""))));
println!("result: {:?}", hex_or_die_trying(None));

result: Ok(15)
result: Ok(10)
result: Err("got empty string")
result: Err("got None")


### while-let

In [9]:
let mut name = String::from("Comprehensive Rust 🦀");
while let Some(c) = name.pop() {
    println!("character: {c}");
}

character: 🦀
character:  
character: t
character: s
character: u
character: R
character:  
character: e
character: v
character: i
character: s
character: n
character: e
character: h
character: e
character: r
character: p
character: m
character: o
character: C


()

# Methods and Traits

## Methods

In [10]:
#[derive(Debug)]
struct Race {
    name: String,
    laps: Vec<i32>,
}

impl Race {
    // No receiver, a static method
    fn new(name: &str) -> Self {
        Self { name: String::from(name), laps: Vec::new() }
    }

    // Exclusive borrowed read-write access to self
    fn add_lap(&mut self, lap: i32) {
        self.laps.push(lap);
    }

    // Shared and read-only borrowed access to self
    fn print_laps(&self, index: bool) {
        println!("Recorded {} laps for {}:", self.laps.len(), self.name);
        if index {
            for (idx, lap) in self.laps.iter().enumerate() {
                println!("Lap {idx}: {lap} sec");
            }
        } else {
            for lap in self.laps.iter() {
                println!("{lap} sec");
            }
        }
    }

    // Exclusive ownership of self
    fn finish(self) {
        let total: i32 = self.laps.iter().sum();
        println!("Race {} is finished, total lap time: {}", self.name, total);
    }
}

In [11]:
let mut race = Race::new("Monaco Grand Prix");
race.add_lap(70);
race.add_lap(68);
race.print_laps(false);
race.add_lap(71);
race.print_laps(true);
race.finish();

Recorded 2 laps for Monaco Grand Prix:
70 sec
68 sec
Recorded 3 laps for Monaco Grand Prix:
Lap 0: 70 sec
Lap 1: 68 sec
Lap 2: 71 sec
Race Monaco Grand Prix is finished, total lap time: 209


## Traits

In [2]:
struct Dog {
    name: String,
    age: i8,
}
struct Cat {
    lives: i8,
}

trait Pet {
    fn talk(&self) -> String;

    fn greet(&self) {
        println!("Oh you're a cutie! What's your name? {}", self.talk());
    }
}

impl Pet for Dog {
    fn talk(&self) -> String {
        format!("Woof, my name is {}!", self.name)
    }
}

impl Pet for Cat {
    fn talk(&self) -> String {
        String::from("Miau!")
    }
}

In [3]:
let captain_floof = Cat { lives: 9 };
let fido = Dog { name: String::from("Fido"), age: 5 };

captain_floof.greet();
fido.greet();

Oh you're a cutie! What's your name? Miau!
Oh you're a cutie! What's your name? Woof, my name is Fido!


## Deriving

In [4]:
#[derive(Debug, Clone, Default)]
struct Player {
    name: String,
    strength: u8,
    hit_points: u8,
}

In [5]:
let p1 = Player::default(); // Default trait adds `default` constructor.
let mut p2 = p1.clone(); // Clone trait adds `clone` method.
p2.name = String::from("EldurScrollz");
// Debug trait adds support for printing with `{:?}`.
println!("{:?} vs. {:?}", p1, p2);

Player { name: "", strength: 0, hit_points: 0 } vs. Player { name: "EldurScrollz", strength: 0, hit_points: 0 }


## Trait Objects

In [6]:
struct Dog {
    name: String,
    age: i8,
}
struct Cat {
    lives: i8,
}

trait Pet {
    fn talk(&self) -> String;
}

impl Pet for Dog {
    fn talk(&self) -> String {
        format!("Woof, my name is {}!", self.name)
    }
}

impl Pet for Cat {
    fn talk(&self) -> String {
        String::from("Miau!")
    }
}

In [12]:
let pets: Vec<Box<dyn Pet>> = vec![
    Box::new(Cat { lives: 9 }),
    Box::new(Dog { name: String::from("Fido"), age: 5 }),
];
for pet in pets {
    println!("Hello, who are you? {}", pet.talk());
}

Hello, who are you? Miau!
Hello, who are you? Woof, my name is Fido!


()

In [13]:
println!("{} {}", std::mem::size_of::<Dog>(), std::mem::size_of::<Cat>());
println!("{} {}", std::mem::size_of::<&Dog>(), std::mem::size_of::<&Cat>());
println!("{}", std::mem::size_of::<&dyn Pet>());
println!("{}", std::mem::size_of::<Box<dyn Pet>>());

32 1
8 8
16
16


# Generic

## Generic Functions

In [40]:
/// Pick `even` or `odd` depending on the value of `n`.
fn pick<T>(n: i32, even: T, odd: T) -> T {
    if n % 2 == 0 {
        even
    } else {
        odd
    }
}

println!("picked a number: {:?}", pick(97, 222, 333));
println!("picked a tuple: {:?}", pick(28, ("dog", 1), ("cat", 2)));

picked a number: 333
picked a tuple: ("dog", 1)


## Generic Functions

In [41]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

impl Point<u32> {
    fn coords(&self) -> (&u32, &u32) {
        (&self.x, &self.y)
    }

    // fn set_x(&mut self, x: T)
}

impl Point<f64> {
    fn coords(&self) -> (&f64, u32, &f64) {
        (&self.x, 0, &self.y)
    }
}

// impl<T> Point<T> {
//     fn coords(&self) -> &T {
//         &self.x + &self.y
//     }
// }

let integer = Point { x: 5, y: 10 };
let float = Point { x: 1.0, y: 4.0 };
println!("{integer:?} and {float:?}");
println!("coords(u32): {:?}", integer.coords());
println!("coords(f64): {:?}", float.coords());

Point { x: 5, y: 10 } and Point { x: 1.0, y: 4.0 }
coords(u32): (5, 10)
coords(f64): (1.0, 0, 4.0)


In [44]:
#[derive(Debug)]
struct Point<T, U> {
    x: T,
    y: U,
}

impl<T, U> Point<T, U> {
    fn coords(&self) -> (&T, &U) {
        (&self.x, &self.y)
    }
}

let mix = Point { x: 2, y: 3.0 };
println!("{mix:?}");
println!("coords: {:?}", mix.coords());

Error: struct defined here, with 2 generic parameters: `T`, `U`

Error: struct defined here, with 2 generic parameters: `T`, `U`

Error: struct defined here, with 2 generic parameters: `T`, `U`

Error: struct defined here, with 2 generic parameters: `T`, `U`

Error: struct defined here, with 2 generic parameters: `T`, `U`

Error: struct defined here, with 2 generic parameters: `T`, `U`